## Building a Language Model Application: Chat Models

- Chat models are a variation on language models. while chat models use language models under the hood, the interface they expose is a bit different: rather that expose a "teext in, text out" API, they expose an interface where "chat messages" are the inputs and outputs. 

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

**NOTE**: LangChain provides many modelues that can be used to build language model application. Modules can be combined to create more compalex application, or be used individually for simple application. 

### Get Message Completions from a Chat Model

- I can get chat complitions by passing one or more messages to chat model and response will be message. 

- The types of message currently supported in Langchain are `AIMessage`, `HumanMessage`, `SystemMessage`, and `ChatMessage` - `Chatmessage` takes in an arbitrary role parrameter. Most of the time, we'll just be dealling with `HumanMessage`, `AIMessage` and `SystemMessage`. 

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(temperature=0)

In [12]:
# we can now get completions by passing in a single message
chat([HumanMessage(content="Translate this sentence from English to Gujarati. I love programming.")])

AIMessage(content='હું પ્રોગ્રામિંગ પસંદ કરું છું.', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 20, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9321712f-71fe-4e92-afea-39ae5501322f-0')

In [13]:
trns.content

'હું પ્રોગ્રામિંગ પસંદ કરું છું.'

In [14]:
# we can pass multiple messages for OpenAI’s gpt-3.5-turbo and gpt-4 models
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Gujarati."),
    HumanMessage(content="Translate this sentence from English to Gujarati. I love programming.")
]
chat(messages)

AIMessage(content='હું પ્રોગ્રામિંગ પસંદ કરું છું.', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 36, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ce1d2fee-0f82-4e39-a4d7-e1430aac0d14-0')

In [15]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Gujarati."),
        HumanMessage(content="Translate this sentence from English to Gujarati. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Gujarati."),
        HumanMessage(content="Translate this sentence from English to Gujarati. I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='હું પ્રોગ્રામિંગ પસંદ કરું છું.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='હું પ્રોગ્રામિંગ પસંદ કરું છું.', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 36, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b68fceb0-f8b3-4507-a9e2-f0cb3ce7b3fb-0'))], [ChatGeneration(text='હું કૃત્રિમ બુધિમત્તાને પ્રેમ કરું છું.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='હું કૃત્રિમ બુધિમત્તાને પ્રેમ કરું છું.', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 37, 'total_tokens': 104}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-089cb22f-f5f7-4eea-907d-37c25f2b4c2f-0'))]], llm_output={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 73, 'total_tokens'

In [16]:
result.llm_output['token_usage']

{'completion_tokens': 120, 'prompt_tokens': 73, 'total_tokens': 193}

## Chat Prompt Templets

- Insted of hard coding that text I want to ask, simillar to LLMs, I can use make of templeting by using a `MessageProptTemplate`

- I can build a `ChatPromptTemplate` from one or more `MessagePromptTemplates`. 

- I can use `ChatPromptTemplate's` `format_prompt` - this return a `PromptValue`, Which Ican convert to a string or `Message` object, depending on whether you want to use the formatted values as input to an llm or chat model.

- for simplicity, there is a `from_template` method exposed on the template which makes our task lot easier. 

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = '{text}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language= "English", output_language="Gujarati", text="I love programming.").to_messages())
     

AIMessage(content='હું પ્રોગ્રામિંગ પસંદ કરું છું.', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 26, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2e08a8c7-b8cf-4cd2-846a-bc808cb4cdb2-0')

## Chains and Chat Models 
- In real applciation, using LLM in isolation is OK for some applicatopn but in most of the cases it requires chaining. And chianging with PtromptTemplate might be a neccessity. 

- A chain in LangChain is made up of links, which can either primitives like LLMs or other chains. 

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt = chat_prompt)
chain.run(input_language="English", output_language = "Gujarati", text = "I love programming.")

'હું પ્રોગ્રામિંગ પસંદ કરું છું.'

## Agents Chat Models

- So, far what we did was to run the chain in predetermined order. 

- Agent can also be used with chat models, you can initialize one using `AgentType`. `CHAT_ZERO_SHOT_REACT_DESCRIPTION` as the agent type.

in order to load agents, understanding the following concepts is crucial. 

- **Tool**: A function that performs a specific duty. This can be thins like: Google Search, database llokpu, Python REMP, Other chains. 

- **ChatModels**: The chat models powering the agent. 

- **Agent**: Agent involve an LLm making decisions about which actions to take, taking that action, seeing an observation, and repeating that until its done. 

**Agent**: For a list supported agents and their specifications.

**Tool**: For a list of predefined tools and their specifications.

In [28]:
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [31]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [38]:
chat = ChatOpenAI(temperature=0)

llm = OpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm = llm)

agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is the president of Finland ? What is 2+2 ?")



> Entering new AgentExecutor chain...
Question: Who is the president of Finland? What is 2+2?
Thought: The first question requires a search, while the second question involves a simple math calculation.
Action:
```
{
  "action": "Search",
  "action_input": "President of Finland"
}
```

Observation: Alexander Stubb
Thought:Action:
```
{
  "action": "Calculator",
  "action_input": "2+2"
}
```
Observation: Answer: 4
Thought:Final Answer: Alexander Stubb, 4

> Finished chain.


'Alexander Stubb, 4'

In [36]:
# !pip install numexpr

## Memory: Add State to Chains and Agents 

- So far, all the chains and agents we've gone through have been stateless. but often, you may want a chain or agent to have some concept of "memory" so that it may remember information about its previous interactions.

- For example, while designing a chatbot you want it to remember previous message or previous several messages.

- Short-term memory

- Long-term-memory (remembering key pieces of information over time)

- I can use Memory with chains and agents initialized with chat models. The main difference between this and Memory for LLMs is that rather than trying to condense all previous messages into a string, I can keep them as their own unique memory object.

In [39]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'

In [40]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That's great to hear! I'm here to chat with you and answer any questions you might have. What's on your mind today?"

In [41]:
conversation.predict(input="Tell me about yourself.")

"I'm an artificial intelligence designed to assist and provide information to users like you. I have been trained on a vast amount of data to help with a wide range of topics, from answering questions to providing recommendations. I'm constantly learning and improving to better serve your needs. Feel free to ask me anything you'd like to know!"

## Conclusion

- The decision to use a chat model or an LLM would depend on the specific task you are trying to accomplish. Chat models are designed to have structured conversations with humans, so they would be useful for tasks like creating chatbots or customer service agents. On the other hand, LLMs are more general language models that can be used for a wide range of tasks, such as language translation, text generation, or summarization.

- In general, if the task involves interacting with humans in a conversational way, a chat model would be the better choice. But if the task involves generating or processing large amounts of text, an LLM would be more appropriate. It's also worth noting that LLMs can be used in conjunction with other models, such as text embedding models, to improve their performance on specific tasks.